# Offer Generation Agent

In [1]:
from pathlib import Path

from langchain.agents import AgentExecutor, tool, create_tool_calling_agent
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv

load_dotenv()

True

## Tools

### Style Guidelines Retriever Tool

In [2]:
text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ],
    strip_headers=False
)
texts = text_splitter.split_text(Path("data/style_guidelines_kb.md").read_text())

In [3]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever(k=3)

In [4]:
style_guidelines_retriever_tool = create_retriever_tool(
    retriever,
    "style_guidelines_retriever_tool",
    "A tool for retrieving guidelines on creating personalized promotional email subjects and contents for customers, emphasizing a personable tone, customer focus, clarity, and a clear call-to-action, while maintaining brand alignment and accessibility. Make sure to search for relevent guidelines around structure, content, subject, etc. before writing the email.",
)

In [5]:
style_guidelines_retriever_tool.invoke("how to write a call to action?")

'### Email Structure  \n1. **Subject Line:**  \n- Capture attention.\n- Highlight the offer or benefit.\n- Keep it concise (50 characters or fewer).  \n**Examples:**  \n- "Stay with Us and Save 20%!"\n- "Thank You for 2 Years! Here’s a Special Gift."  \n2. **Greeting:**  \n- Use the customer’s name for personalization.  \n**Examples:**  \n- "Hi Alex,"\n- "Dear Maria,"  \n3. **Introduction:**  \n- Acknowledge the customer’s relationship with the company.\n- Set the stage for the offer.  \n**Examples:**  \n- "We appreciate you being a valued customer for the past 18 months."\n- "Thank you for your continued loyalty."  \n4. **Body:**  \n- Highlight the promotion and its benefits.\n- Tailor the message to the customer’s attributes (e.g., churn likelihood, usage).\n- Provide a reason for the offer (e.g., anniversary, feedback, loyalty).  \n**Examples:**  \n- "We noticed you’ve been making the most of your plan. As a thank-you, we’re offering you 20% off your bill for the next six months."\n

### Generate Email Tool

In [6]:
# class GenerateEmailInput(BaseModel):
#     customer_insight_report: dict = Field(
#         description="A customer insight report that includes information on usage, charges, customer service calls, churn probability, and more.",
#         default=None,
#     )
#     available_promotions: list = Field(
#         description="List of promotional offers available for this customer",
#         default=None,
#     )
#     style_guidelines: str = Field(
#         description="Relevant style guidelines for generating the personalized promotional email",
#         default=None,
#     )

class GenerateEmailInput(BaseModel):
    style_guidelines: str = Field(
        description="Relevant style guidelines for generating the personalized promotional email. Make sure to include the entirety of the relevant style guidelines.",
        default=None,
    )


@tool("generate_email_tool", args_schema=GenerateEmailInput)
def generate_email_tool(
    style_guidelines: str
):
    """
    Tool to generate a personalized promotional email based on customer data, promotions, and style guidelines.
    Make sure to include the entirety of the relevant style guidelines.
    """
    
    llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        template="""
        Generate a personalized promotional email subject and content based on the following customer data, available promotions, and style guidelines.
        Choose the single most relevant promotion based on the customer information.
        Be sure to adhere to the style guidelines.
        
        ## Customer data
        {customer_insight_report}
        
        ## Style guidelines
        {style_guidelines}
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(
        {
            "customer_insight_report" : customer_insight_report,
            "style_guidelines" : style_guidelines
        }
    )

### Revise Email Tool

In [7]:
# class GenerateEmailInput(BaseModel):
#     customer_insight_report: dict = Field(
#         description="A customer insight report that includes information on usage, charges, customer service calls, churn probability, and more.",
#         default=None,
#     )
#     available_promotions: list = Field(
#         description="List of promotional offers available for this customer",
#         default=None,
#     )
#     style_guidelines: str = Field(
#         description="Relevant style guidelines for generating the personalized promotional email",
#         default=None,
#     )

class ReviseEmailInput(BaseModel):
    generated_email: str = Field(
        description="Generated promotional email",
        default=None,
    )
    simulated_customer_feedback: str = Field(
        description="Simulated customer feedback to improve the email effetiveness.",
        default=None,
    )
    style_guidelines: str = Field(
        description="Relevant style guidelines for generating the personalized promotional email",
        default=None,
    )


@tool("revise_email_tool", args_schema=ReviseEmailInput)
def revise_email_tool(
    generated_email: str,
    simulated_customer_feedback: str,
    style_guidelines: str
):
    """
    Tool to revise personalized promotional email based on customer data, promotions, style guidelines
    and revised customer input.
    """
    
    llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        template="""
        Revise the personalized promotional email based on customer data, promotions, style guidelines
        and revised customer input. Update the email to better reflect the simulated customer
        feedback. Ensure that the changes do not change the factual accuracy
        of the available promotions or adherence to style guidelines.
        
        ## Generated email
        {generated_email}
        
        ## Customer data
        {customer_insight_report}
        
        ## Simulated customer feedback
        {simulated_customer_feedback}
        
        ## Style guidelines
        {style_guidelines}
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(
        {
            "email_subject" : email_subject,
            "customer_insight_report" : customer_insight_report,
            "style_guidelines" : style_guidelines,
            "simulated_customer_feedback" : simulated_customer_feedback
        }
    )

### Customer Simulation Tool

In [8]:
class CustomerSimulationInput(BaseModel):
    generated_email: str = Field(
        description="Generated promotional email",
        default=None,
    )


@tool("customer_simulation_tool", args_schema=CustomerSimulationInput)
def customer_simulation_tool(generated_email: str):
    """
    Tool to see how well the customer will respond to the email - use this to improve the relevance and tone.
    """
    
    llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        template="""
        Act as the customer described in the below customer insight report. How successful
        would this email be? Describe the effectiveness and potential improvements in a few short sentences.
        
        ## Customer data
        {customer_insight_report}
        
        ## Generated email
        {generated_email}
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(
        {
            "generated_email" : generated_email,
            "customer_insight_report" : customer_insight_report,
        }
    )

## Agent

In [9]:
from langchain.globals import set_debug

set_debug(False)

In [10]:
tools = [
    style_guidelines_retriever_tool,
    generate_email_tool,
    revise_email_tool,
    customer_simulation_tool
]

In [11]:
llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)

In [12]:
from langchain_core.tools.render import ToolsRenderer, render_text_description

In [13]:
OLD_PROMPT = """
Generate a personalized promotional email from IGZ Telecom for the customer using the following flow:
- Use the style_guidelines_retriever_tool for email structure, tone, and reference examples.
- Use the generate_email_tool to generate the promotional email considering the style guidelines and customer information.
- Use the customer_simulation_tool to ensure the email's relevance and get feedback.
- Use the style_guidelines_retriever_tool for guidance on updating the email with customer feedback.
- Use the revise_email_tool to revise and update the email as needed.
- Repeat the above steps as needed to create a high quality and relevant promotional email for the customer.
- When you are satisfied with the email, output it without any pre or post amble.
"""
# Note: always use the customer_simulation_tool and revise_email_tool to make sure the email will be effective.
# OLD_PROMPT = """
# Your goal is to generate a personalized and highly effective promotional email for the customer. To achieve this, follow these steps:  

# 1. Start by using the `calculate_promotion_tool` to identify the most relevant and attractive offers for the customer.  
# 2. Retrieve style guidelines (structure, tone, and examples) using the `style_guidelines_retriever_tool` to ensure the email matches the desired tone and format.  
# 3. Draft the initial email using the `generate_email_tool`, incorporating the selected offers and style guidelines.  

# 4. **Simulate and Improve**:  
#    - Use the `customer_simulation_tool` to simulate how the customer might perceive and respond to the email. Pay attention to areas where the email could be more engaging or relevant.  
#    - Use the `revise_email_tool` to make improvements based on the simulation feedback.  

# 5. **Iterate if Needed**:  
#    - Repeat the simulation and revision process as many times as necessary to ensure the email is compelling, personalized, and effective.  

# 6. When you are satisfied with the email, output only the final version. Do not include any additional text or explanation.  

# **Important Notes:**  
# - Always use the `customer_simulation_tool` at least once to validate relevance and effectiveness.  
# - Prioritize creating an email that is both engaging and aligned with the customer's preferences and needs.  

# """

In [14]:
prompt = ChatPromptTemplate.from_messages(
            [
                ("system", OLD_PROMPT),
                ("user", "{input}"),
                ("ai", "{agent_scratchpad}"),
            ]
        )

agent = AgentExecutor(
    agent=create_tool_calling_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
    ),
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [15]:
customer_insight_report = {
    'user_id': 460,
    'account_length': '6 years, 7 months',
    'total_charge_usage': 'medium-low',
    'total_minutes_usage': 'medium',
    'number_customer_service_calls': 5,
    'sentiment_label': 'neutral',
    'account_length_months': 79,
    'churn_likelihood_percentage': 0.464,
    'support_ticket_summary': 'The customer is concerned about the escalating cost of voice services impacting turnover and is looking for alternative options, but hangs up before the TelCom agent can provide further assistance.',
    'available_promotions': [
        'Service Credit: $10 off next bill',
        'Bundle Offer: Add premium channels for $5/month',
        'Anniversary Offer: 1 month free service'
    ]
}

In [16]:
customer_insight_report = {
    'user_id': 2296,
    'account_length': '9 years, 9 months',
    'total_charge_usage': 'medium',
    'total_minutes_usage': 'medium',
    'number_customer_service_calls': 2,
    'sentiment_label': 'neutral',
    'account_length_months': 117,
    'churn_likelihood_percentage': 0.01,
    'support_ticket_summary': 'The customer is experiencing slow performance on their phone and is requesting an upgrade, with a technician scheduled to come to their home for the upgrade appointment.',
    'available_promotions': [
        'Bundle Offer: Add premium channels for $5/month',
        'Anniversary Offer: 1 month free service'
    ]
}

In [17]:
customer_insight_report = {
    'user_id': 1902,
    'account_length': '10 years, 5 months',
    'total_charge_usage': 'high',
    'total_minutes_usage': 'high',
    'number_customer_service_calls': 1,
    'sentiment_label': 'negative',
    'account_length_months': 125,
    'churn_likelihood_percentage': 0.792,
    'support_ticket_summary': 'Customer is frustrated with the poor customer service they have been receiving for their phone, despite not having any issues with the phone itself.',
    'available_promotions': [
        'Retention Offer: 20% discount for 6 months',
        'Loyalty Offer: Free international minutes',
        'Anniversary Offer: 1 month free service'
    ]
}

In [18]:
email = agent.invoke(
    input={"input" : "Write a promotional email for Peggy"},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `style_guidelines_retriever_tool` with `{'query': 'personalized promotional email for Peggy'}`


**IGZ Telecom Style Guide for Generating Promotional Email Offers**

### Purpose  
This guide provides guidelines for crafting effective, personalized promotional email offers for customers. Emails should be engaging, clear, and reflect the specific promotional offers and customer attributes. Use this guide to ensure consistency in tone, language, and structure.  
---

### General Principles  
1. **Personable Tone:** Emails should feel conversational and friendly. Avoid overly formal language but maintain professionalism.
2. **Customer Focus:** Incorporate details specific to the customer, such as their usage patterns, account history, and sentiment profile.
3. **Clarity:** Ensure the email is easy to read. Avoid jargon and long, complex sentences.
4. **Call-to-Action:** Include a clear and compelling call-to-action (CTA) encouraging the customer to take advantage of the offer.
5

In [19]:
print(email["output"])

Subject: Exclusive Offer for Our Valued Customer!

Hi Peggy,

We hope this email finds you well! We've noticed that you've been with us for 10 years and 5 months, and we truly appreciate your loyalty. We understand that you've had a frustrating experience with our customer service recently, and we want to make it right for you.

As a token of our appreciation, we're excited to offer you a special Retention Offer: 20% discount for 6 months on your service. We value your feedback and want to ensure you have a positive experience with us moving forward.

Don't miss out on this exclusive offer tailored just for you. Simply click the link below to redeem your discount and continue enjoying our high-quality service.

Claim your offer now!

We're here to assist you every step of the way. If you have any questions or need assistance, feel free to reach out to our customer support team.

Thank you for being a valued member of our community.

Warm regards,
IGZ Telecom Team
